In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()

In [6]:
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
# Save references to each table
measurement = Base.classes.measurement
station= Base.classes.station

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [10]:
# Find the most recent date in the data set in measurment table. 
recent = session.query(measurement.date).order_by(measurement.date.desc()).first()
print(recent)

('2017-08-23',)


In [11]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Part 1: Starting from the most recent data point in the database. 
print(dt.date(2017, 8 ,23))

# Calculate the date one year from the last date in data set.
year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print(year_ago)


2017-08-23
2016-08-23


In [12]:
# Part 2: Perform a query to retrieve the data and precipitation scores
station_results = session.query(measurement.station, func.count(measurement.station)).\
            group_by(measurement.station).\
            order_by(func.count(measurement.station).desc()).all()
station_results

[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

In [ ]:
# Part 3: Perform a query to retrieve the data and precipitation scores
prcp_score = session.query(measurement.date, measurement.prcp).\
                    filter(measurement.date >= '2016-08-23').\
                    filter(measurement.date <= '2017-08-23').all()
                                  
# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_score_df = pd.DataFrame(prcp_score, columns=['date', 'prcp'])
prcp_score_df.set_index('date', inplace=True)

# Sort the dataframe by date
sorted_prcp = prcp_score_df.sort_index()
sorted_prcp.head()

In [ ]:
#################FIX GRAPH LAYOUT Tick size############################################
# Use Pandas Plotting with Matplotlib to plot the data
date = sorted_prcp.index.values
rain = sorted_prcp["prcp"]

plt.figure(figsize=(20,3))
plt.bar(date, rain, color='b', alpha=0.65, align="center")
plt.xlim(0, 365)
plt.ylim(0,7)
plt.xticks([])

plt.title("Hawaiian Percipitation Records")
plt.xlabel("Date")
plt.ylabel("Precipitation Measurement (inch)")  
plt.tight_layout()
plt.savefig("Hawaiian Percipitation Records_v1.png")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_mean = sorted_prcp["prcp"].mean().round(3)
prcp_median = sorted_prcp["prcp"].median().round(3)
prcp_var = sorted_prcp["prcp"].var().round(3)
prcp_stdev = sorted_prcp["prcp"].std().round(3)
prcp_min = sorted_prcp["prcp"].min()
prcp_max = sorted_prcp["prcp"].max()

print(f'Precipitation statistics for Hawaii mean {prcp_mean}", median {prcp_median}", variance {prcp_var}",\
 standard deviation {prcp_stdev}", minimum {prcp_min}", and maximum {prcp_max}"')

In [ ]:
####ALTERNATIVE CODE FOR STATISTICAL ANALYSIS######
sorted_prcp.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_count = session.query(station).count()
print(station_count)

####ALTERNATIVE CODE FOR UNIQUE STATION###### 
unique_number = session.query(measurement).group_by(measurement.station).count()
print(unique_number)

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
unique_stations = session.query(measurement.station, func.count(measurement.station)).\
group_by(measurement.station).\
order_by(func.count(measurement.station).desc()).all()

unique_stations

In [ ]:
#Calculate the lowest, highest, and average temperature for most frequently reporting station USC00519281.
#Hint: You will need to use a function such as func.min, func.max, func.avg, and func.count in your queries.
temp_results = session.query(func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs)).\
filter(measurement.station == "USC00519281").all()
temp_results


In [ ]:
#####ALTERNATIVE METHOD FOR STATISTICAL METHOD######
temperature = [measurement.id, measurement.station, measurement.date, measurement.prcp, measurement.tobs]

station_temp = session.query(*temperature)
station_temp_df = pd.DataFrame(station_temp, columns=['id', 'station', 'date', 'prcp', 'temp'])
station_temp_df.set_index('id', inplace=True)

waihee_station = station_temp_df.loc[station_temp_df["station"]=="USC00519281"]

temp_min = waihee_station["temp"].min()
temp_max = waihee_station["temp"].max()
temp_avg = waihee_station["temp"].mean().round(1)
print(f'Waihee station minimum temp is {temp_min}F, the maximum temp is {temp_max}, and the average temp is {temp_avg}F')


In [ ]:
# Using the most active station id "station USC00519281"
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
annualized = session.query(measurement.id, measurement.station, measurement.date, measurement.tobs).\
filter(measurement.date > '2016-08-23').\
filter(measurement.date < '2017-08-23').\
filter(measurement.station == "USC00519281").all()

#Create a dataframe from filtered query
waihee_annual = pd.DataFrame(annualized, columns=['id','station', 'date', 'temp'])
waihee_annual.set_index('id', inplace=True)
waihee_annual.head()

In [ ]:
#plot the data in a historgram
temperature = waihee_annual["temp"]
plt.hist(temperature, bins=12, edgecolor='xkcd:light gray', linewidth=1,)
plt.title("Temperature Recordings for Waihee Station")
plt.xlabel('Temperature (°F)')
plt.ylabel("Frequency")
plt.show()

# Close session

In [ ]:
# Close Session
session.close()